In [1]:
# We are using black as a code formatter
%load_ext nb_black

<IPython.core.display.Javascript object>

In [37]:
import requests
import json
import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [3]:
# pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

<IPython.core.display.Javascript object>

In [4]:
# the UN's API includes many entities which are not sovereign states,
# yet are listed as "Country". This workaround is a manuel fix for if
# one is only interested in countries by the classic definition
not_countries = [
    "American Samoa",
    "Bermuda",
    "British Virgin Islands",
    "Cayman Islands",
    "Mayotte",
    "Cook Islands",
    "Faroe Islands",
    "Falkland Islands (Malvinas)",
    "French Guiana",
    "French Polynesia",
    "Gibraltar",
    "Greenland",
    "Guadeloupe",
    "Guam",
    "China, Hong Kong SAR",
    "China, Macao SAR",
    "Martinique",
    "Montserrat",
    "Curaçao",
    "Aruba",
    "Sint Maarten (Dutch part)",
    "Bonaire, Sint Eustatius and Saba",
    "New Caledonia",
    "Niue",
    "Northern Mariana Islands",
    "Puerto Rico",
    "Réunion",
    "Saint Helena",
    "Anguilla",
    "Saint Pierre and Miquelon",
    "Tokelau",
    "Turks and Caicos Islands",
    "Isle of Man",
    "United States Virgin Islands",
    "Wallis and Futuna Islands",
]

<IPython.core.display.Javascript object>

#### The base path for accessing the API is:
base_path ["https://population.un.org/dataportalapi/api/v1"](https://population.un.org/dataportalapi/api/v1)

#### Most common reported status codes
* 200 : Successful request
* 400 : Bad request
* 404 : Input parameters not found
* 406 : Requested output format not allowed
* 500 : Server error

#### Structure of API response (json)

* pageNumber : the current page of the response, which may have multiple pages
* pageSize : the number of records returned on the current page (a maximum of 100 records will be returned)
* previousPage : the path to the previous page of the response when multiple pages are returned
* nextPage : the path to the next page of the response when multiple pages are returned
* pages : the total number of pages in the response
* total : the total number of records in the response
* data : the actual data returned in the response

### Indicators

In [70]:
# There are 82 indicators
base_url_indicator = "https://population.un.org/dataportalapi/api/v1/indicators"
response = requests.get(base_url_indicator).json()
print("Response:", json.dumps(response, indent=2))

# response_data = json.dumps(response["data"], indent=2)
# print("Response indicators data:", response_data)

Response: {
  "pageNumber": 1,
  "pageSize": 100,
  "previousPage": null,
  "nextPage": null,
  "pages": 1,
  "total": 60,
  "data": [
    {
      "id": 1,
      "name": "Contraceptive prevalence: Any method (Percent)",
      "shortName": "CPAnyP",
      "description": "Percentage of women of reproductive age (15-49 years) who are currently using any method of contraception",
      "displayName": "Any",
      "dimAge": false,
      "dimSex": false,
      "dimVariant": true,
      "dimCategory": true,
      "defaultAgeId": 31,
      "defaultSexId": 2,
      "defaultVariantId": 4,
      "defaultCategoryId": 100,
      "variableType": "relative",
      "valueType": "percent",
      "unitScaling": 0.01,
      "precision": 1,
      "isThousandSeparatorSpace": false,
      "formatString": "#0.0",
      "unitShortLabel": "%",
      "unitLongLabel": "per cent",
      "nClassesDefault": 5,
      "downloadFileName": "PercentageContraceptive_AnyMethod",
      "sourceId": 23,
      "sourceName": "

<IPython.core.display.Javascript object>

### Topics

In [21]:
# There are 10 topics
base_url_topic = "https://population.un.org/dataportalapi/api/v1/topics"
response = requests.get(base_url_topic).json()
# print("Response:", json.dumps(response, indent=2))
# response_data = json.dumps(response["data"], indent=2)
# print("Response topics data:", response_data)
df = pd.json_normalize(response["data"])
df

,id,name,shortName,sortOrder
0,0,Not applicable,NA,0
1,1,Population,Pop,1
2,2,Fertility,Fert,2
3,3,Mortality,Mort,3
4,4,International Migration,iMigration,4
5,5,Family Planning,FP,5
6,6,Marital Status,MarStat,6
7,7,All Components,All,7
8,8,Child Mortality,IGME,8
9,9,Maternal Mortality,MMEIG,9


<IPython.core.display.Javascript object>

### Locations

In [10]:
# There are X locations
base_url_location = "https://population.un.org/dataportalapi/api/v1/locations?pageNumber=3&pageSize=1000"
response = requests.get(base_url_location).json()
# print("Response:", json.dumps(response, indent=2))
response_data = json.dumps(response["data"], indent=2)
print("Response locations data:", response_data)

Response locations data: [
  {
    "id": 756,
    "parentId": 926,
    "name": "Switzerland",
    "iso3": "CHE",
    "iso2": "CH",
    "locationTypeId": 4,
    "locationType": "Country",
    "longitude": 8.22751235961914,
    "latitude": 46.81818771362305
  },
  {
    "id": 760,
    "parentId": 922,
    "name": "Syrian Arab Republic",
    "iso3": "SYR",
    "iso2": "SY",
    "locationTypeId": 4,
    "locationType": "Country",
    "longitude": 38.9968147277832,
    "latitude": 34.80207443237305
  },
  {
    "id": 762,
    "parentId": 5500,
    "name": "Tajikistan",
    "iso3": "TJK",
    "iso2": "TJ",
    "locationTypeId": 4,
    "locationType": "Country",
    "longitude": 71.27609252929688,
    "latitude": 38.86103439331055
  },
  {
    "id": 764,
    "parentId": 920,
    "name": "Thailand",
    "iso3": "THA",
    "iso2": "TH",
    "locationTypeId": 4,
    "locationType": "Country",
    "longitude": 100.99253845214844,
    "latitude": 15.87003231048584
  },
  {
    "id": 768,
    "pare

<IPython.core.display.Javascript object>

### Locations With Aggregates

In [11]:
# There are x aggregate locations
base_url_aggregate = "https://population.un.org/dataportalapi/api/v1/locationsWithAggregates?pageNumber=1&page"
response = requests.get(base_url_aggregate).json()
print("Response locations with aggregates:", json.dumps(response, indent=2))
# response_data = json.dumps(response["data"], indent=2)
# print(response_data)

Response locations with aggregates: [
  {
    "Id": 4,
    "Name": "Afghanistan",
    "Iso2": "AF",
    "Iso3": "AFG",
    "Longitude": 67.70995330810547,
    "Latitude": 33.939109802246094,
    "Region": "Asia",
    "SubRegion": "Southern Asia",
    "SpecialGrouping": "Landlocked developing countries (LLDCs)",
    "WorldBankIncomeGroup": "Low-income countries",
    "UNDevelopmentGroup": "Least developed countries",
    "SDGRegion": "Central and Southern Asia"
  },
  {
    "Id": 8,
    "Name": "Albania",
    "Iso2": "AL",
    "Iso3": "ALB",
    "Longitude": 20.16833114624023,
    "Latitude": 41.1533317565918,
    "Region": "Europe",
    "SubRegion": "Southern Europe",
    "WorldBankIncomeGroup": "Upper-middle-income countries",
    "UNDevelopmentGroup": "Developed regions",
    "SDGRegion": "Europe and Northern America"
  },
  {
    "Id": 12,
    "Name": "Algeria",
    "Iso2": "DZ",
    "Iso3": "DZA",
    "Longitude": 1.659626007080078,
    "Latitude": 28.033885955810547,
    "Region":

<IPython.core.display.Javascript object>

### Sources

In [12]:
# there are 26 sources
base_url_source = "https://population.un.org/dataportalapi/api/v1/sources?sort=id"
response = requests.get(base_url_source).json()
# print("Response source:", json.dumps(response, indent=2))
response_data = json.dumps(response["data"], indent=2)
print(response_data)

[
  {
    "id": 1,
    "name": "World Urbanization Prospects",
    "sourceYear": 2017,
    "startYear": 1950,
    "endYear": 2017,
    "citation": "United Nations, Department of Economic and Social Affairs, Population Division (2017). World Population Prospects: The 2017 Revision, custom data acquired via website.",
    "url": "https://population.un.org/wup/",
    "author": "United Nations Population Division"
  },
  {
    "id": 2,
    "name": "World Population Prospects",
    "sourceYear": 2018,
    "startYear": 1950,
    "endYear": 2018,
    "citation": "United Nations, Department of Economic and Social Affairs, Population Division (2018). World Urbanization Prospects: The 2018 Revision, custom data acquired via website.",
    "url": "https://population.un.org/wpp/",
    "author": "United Nations Population Division"
  },
  {
    "id": 3,
    "name": "World Contraceptive Use",
    "sourceYear": 2017,
    "startYear": 1970,
    "endYear": 2017,
    "citation": "United Nations, Departm

<IPython.core.display.Javascript object>

### Example 1: Returning a list of indicators

In [13]:
# Declares the base url for calling the API
base_url = "https://population.un.org/dataportalapi/api/v1"

# Creates the target URL, indicators, in this instance
target = base_url + "/indicators/"

# Get the response, which includes the first page of data as well as information on pagination and number of records
response = requests.get(target)

# Converts call into JSON
j = response.json()

# Converts JSON into a pandas DataFrame.
df = pd.json_normalize(
    j["data"]
)  # pd.json_normalize flattens the JSON to accomodate nested lists within the JSON structure

# Loop until there are new pages with data
while j["nextPage"] != None:
    # Reset the target to the next page
    target = j["nextPage"]

    # call the API for the next page
    response = requests.get(target)

    # Convert response to JSON format
    j = response.json()

    # Store the next page in a data frame
    df_temp = pd.json_normalize(j["data"])

    # Append next page to the data frame
    df = pd.concat([df, df_temp], ignore_index=True)

df_indicators_by_topic = df[
    (df["topicShortName"] == "Pop")
    | (df["topicShortName"] == "Fert")
    | (df["topicShortName"] == "Mort")
]
df_indicators_by_topic

,id,name,shortName,description,displayName,dimAge,dimSex,dimVariant,dimCategory,defaultAgeId,defaultSexId,defaultVariantId,defaultCategoryId,variableType,valueType,unitScaling,precision,isThousandSeparatorSpace,formatString,unitShortLabel,unitLongLabel,nClassesDefault,downloadFileName,sourceId,sourceName,sourceYear,sourceStartYear,sourceEndYear,sourceCitation,sourceUrl,topicId,topicName,topicShortName
12,17,Fertility rates by age of mother (5-year),ASFR5,Age-specific fertility rates (ASFR),Fertility rates by age of mother (5-year),True,False,True,False,67,3,4,0,relative,number,1.0,1,False,#0.0,thousands,per thousand,5,Age-specific fertility rates by age of mother,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/,2,Fertility,Fert
13,18,Mean age of childbearing (5-year),MAC5,Mean age of childbearing,Mean age of childbearing (5-year),False,False,True,False,188,3,4,0,duration,number,1.0,1,False,#0.0,years,years,5,Mean age of childbearing,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/,2,Fertility,Fert
14,19,Total fertility rate,TFR5,Total fertility (children per woman),Total fertility rate,False,False,True,False,188,3,4,0,relative,number,1.0,2,False,#0.00,per woman,per woman,5,Total fertility rates,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/,2,Fertility,Fert
18,41,Female population of reproductive age (15-49 years),FemPop15_49,Female population of reproductive age (15-49 years),Female population of reproductive age (15-49 years),False,False,True,False,31,2,4,99,absolute,number,1.0,0,True,### ### ##0,persons,persons,5,FemalePop_reproductive_age,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/,1,Population,Pop
23,46,Population by 5-year age groups and sex,PopByAge5AndSex,"Annual population by five-year age groups and by sex (interpolated data based on 5-year periods). De facto population as of 1 July of the year indicated classified by five-year age groups (0-4, 5-9, 10-14,.., 95-99, 100+).",Annual population by 5-year age groups and by sex,True,True,True,False,7,3,4,0,absolute,number,1.0,0,True,### ### ##0,persons,persons,5,PopBy5YrAgeAndSex,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/,1,Population,Pop
24,47,Population by 1-year age groups and sex,PopByAge1AndSex,"Annual population by single age and by sex (interpolated data based on 5-year age groups and 5-year periods). De facto population as of 1 July of the year indicated classified by single age (0, 1, 2,.., 99, 100+).",Annual population by 1-year age groups and by sex,True,True,True,False,42,3,4,0,absolute,number,1.0,0,True,### ### ##0,persons,persons,5,PopBySingleAgeAndSex,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/,1,Population,Pop
26,49,Total population by sex,TPopulation,Total number of persons by sex (mid-year),Total population by sex,False,True,True,False,

<IPython.core.display.Javascript object>

### Example 2: Returning a list of geographical areas

In [14]:
# Creates the target URL, indicators, in this instance
target = base_url + "/locations/"

# Get the response, which includes the first page of data as well as information on pagination and number of records
response = requests.get(target)

# Converts call into JSON
j = response.json()

# Converts JSON into a pandas DataFrame.
df = pd.json_normalize(
    j["data"]
)  # pd.json_normalize flattens the JSON to accomodate nested lists within the JSON structure

# Loop until there are new pages with data
while j["nextPage"] != None:
    # Reset the target to the next page
    target = j["nextPage"]

    # call the API for the next page
    response = requests.get(target)

    # Convert response to JSON format
    j = response.json()

    # Store the next page in a data frame
    df_temp = pd.json_normalize(j["data"])

    # Append next page to the data frame
    df = pd.concat([df, df_temp], ignore_index=True)

df_countries = df[(df["locationType"] == "Country")]
df_countries

,id,parentId,name,iso3,iso2,locationTypeId,locationType,longitude,latitude
0,4,5501,Afghanistan,AFG,AF,4,Country,67.709953,33.939110
1,8,925,Albania,ALB,AL,4,Country,20.168331,41.153332
2,12,912,Algeria,DZA,DZ,4,Country,1.659626,28.033886
3,16,957,American Samoa,ASM,AS,4,Country,-170.696182,-14.306021
4,20,925,Andorra,AND,AD,4,Country,1.521801,42.506287
...,...,...,...,...,...,...,...,...,...
226,862,931,Venezuela (Bolivarian Republic of),VEN,VE,4,Country,-66.589729,6.423750
227,876,957,Wallis and Futuna Islands,WLF,WF,4,Country,-178.116501,-14.293800
228,882,957,Samoa,WSM,WS,4,Country,-172.104630,-13.759029
229,887,922,Yemen,YEM,YE,4,Country,48.516388,15.552727


<IPython.core.display.Javascript object>

### Example 3: Returning a list of topics

In [15]:
# Creates the target URL, indicators, in this instance
target = base_url + "/topics/"

# Get the response, which includes the first page of data as well as information on pagination and number of records
response = requests.get(target)

# Converts call into JSON
j = response.json()

# Converts JSON into a pandas DataFrame.
df = pd.json_normalize(
    j["data"]
)  # pd.json_normalize flattens the JSON to accomodate nested lists within the JSON structure

# Loop until there are new pages with data
while j["nextPage"] != None:
    # Reset the target to the next page
    target = j["nextPage"]

    # call the API for the next page
    response = requests.get(target)

    # Convert response to JSON format
    j = response.json()

    # Store the next page in a data frame
    df_temp = pd.json_normalize(j["data"])

    # Append next page to the data frame
    df = pd.concat([df, df_temp], ignore_index=True)

df

,id,name,shortName,sortOrder
0,0,Not applicable,NA,0
1,1,Population,Pop,1
2,2,Fertility,Fert,2
3,3,Mortality,Mort,3
4,4,International Migration,iMigration,4
5,5,Family Planning,FP,5
6,6,Marital Status,MarStat,6
7,7,All Components,All,7
8,8,Child Mortality,IGME,8
9,9,Maternal Mortality,MMEIG,9


<IPython.core.display.Javascript object>

### Example 4: Returning a list of sources

In [16]:
# Creates the target URL, indicators, in this instance
target = base_url + "/sources/"

# Get the response, which includes the first page of data as well as information on pagination and number of records
response = requests.get(target)

# Converts call into JSON
j = response.json()

# Converts JSON into a pandas DataFrame.
df = pd.json_normalize(
    j["data"]
)  # pd.json_normalize flattens the JSON to accomodate nested lists within the JSON structure

# Loop until there are new pages with data
while j["nextPage"] != None:
    # Reset the target to the next page
    target = j["nextPage"]

    # call the API for the next page
    response = requests.get(target)

    # Convert response to JSON format
    j = response.json()

    # Store the next page in a data frame
    df_temp = pd.json_normalize(j["data"])

    # Append next page to the data frame
    df = pd.concat([df, df_temp], ignore_index=True)

df

,id,name,sourceYear,startYear,endYear,citation,url,author
0,1,World Urbanization Prospects,2017,1950,2017,"United Nations, Department of Economic and Social Affairs, Population Division (2017). World Population Prospects: The 2017 Revision, custom data acquired via website.",https://population.un.org/wup/,United Nations Population Division
1,2,World Population Prospects,2018,1950,2018,"United Nations, Department of Economic and Social Affairs, Population Division (2018). World Urbanization Prospects: The 2018 Revision, custom data acquired via website.",https://population.un.org/wpp/,United Nations Population Division
2,3,World Contraceptive Use,2017,1970,2017,"United Nations, Department of Economic and Social Affairs, Population Division (2017). World Contraceptive Use 2017 (POP/DB/CP/Rev2017), custom data acquired via website.",http://www.un.org/en/development/desa/population/publications/dataset/contraception/wcu2017.shtml,United Nations Population Division
3,4,Estimates and Projections of Family Planning Indicators,2017,1970,2030,"United Nations, Department of Economic and Social Affairs, Population Division (2017). Model-based Estimates and Projections of Family Planning Indicators 2017, custom data acquired via website.",http://www.un.org/en/development/desa/population/theme/family-planning/cp_model.shtml,United Nations Population Division
4,5,World Fertility Data,2017,1950,2017,"United Nations, Department of Economic and Social Affairs, Population Division (2017). World Fertility Data 2017 (POP/DB/Fert/Rev2017), custom data acquired via website.",http://www.un.org/en/development/desa/population/publications/dataset/fertility/wfd2015.shtml,United Nations Population Division
5,6,World Marriage Data,2017,1950,2017,"United Nations, Department of Economic and Social Affairs, Population Division (2017). World Marriage Data 2017 (POP/DB/Marr/Rev2017), custom data acquired via website.",http://www.un.org/en/development/desa/population/theme/marriage-unions/WMD2015.shtml,United Nations Population Division
6,7,Estimates and Projections of the Number of Women Aged 15-49 Who Are Married or in a Union,2015,1970,2030,"United Nations, Department of Economic and Social Affairs, Population Division (2016). Estimates and Projections of the Number of Women Aged 15-49 Who Are Married or in a Union: 2016 Revision. New York: United Nations.",http://www.un.org/en/development/desa/population/theme/marriage-unions/marriage_estimates.shtml,United Nations Population Division
7,8,Levels & Trends in Child Mortality Report 2015,2015,1990,2015,"UN IGME (2015). Levels and Trends in Child Mortality: Report 2015. Estimates developed by the UN Inter-agency Group for Child Mortality Estimation (United Nations Children's Fund, World Health Organization, World Bank Group and United Nations Population Division of the Department of Economic and Social Affairs), New York: UNICEF.",http://childmortality.org/,United Nations Inter-agency Group for Child Mortality Estimation
8,9,Trends in maternal mortality: 1990 to 2015,2015,1990,2015,"UN MMEIG (2015). Trends in maternal mortality: 1990 to 2015. Estimates by WHO, UNICEF, UNFPA, World Bank Group and the United Nations Population Division, Geneva: WHO.",http://www.who.int/reproductivehealth/publications/monitoring/maternal-mortality-2015/en/,United Nations Inter-agency Group for Child Mortality Estimation
9,10,Levels & Trends in Child Mortality Report 2018,2018,1990,2017,"UN IGME (2018). Levels and Trends in Child Mortality: Report 2018. Estimates developed by the UN Inter-agency Group for Child Mortality Estimation (United Nations Children's Fund, World Health Organization, World Bank Group and United Nations Population Division of the Department of Economic and Social Affairs), New York: UNICEF.",http://childmortality.org/,United Nations Inter-agency Group for Child Mortality Estimation


<IPython.core.display.Javascript object>

### Example 5: Returning data on multiple indicators and geographical areas

In [61]:
# Define a function that will take a relative path as an input, call the API, and return a dataframe
def callAPI(relative_path: str, topic_list: bool = False) -> pd.DataFrame:
    base_url = "https://population.un.org/dataportalapi/api/v1"
    target = (
        base_url + relative_path
    )  # Query string parameters may be appended here or directly in the provided relative path
    # Calls the API
    response = requests.get(target)
    # Reformats response into a JSON object
    j = response.json()
    # The block below will deal with paginated results.
    # If results not paginated, this will be skipped.
    try:
        # If results are paginated, they are transformed into a python dictionary.
        # The data may be accessed using the 'data' key of the dictionary.
        df = pd.json_normalize(j["data"])
        # As long as the nextPage key of the dictionary contains an address for the next API call, the function will continue to call the API and append the results to the dataframe.
        while j["nextPage"] is not None:
            response = requests.get(j["nextPage"])
            j = response.json()
            df_temp = pd.json_normalize(j["data"])
            df = pd.concat([df, df_temp], ignore_index=True)
    except:
        if topic_list:
            df = pd.json_normalize(j)
        else:
            df = pd.DataFrame(j)
    return df


df

,id,name,unit,sourceNote,sourceOrganization,topics,source.id,source.value
0,IC.FRM.FEMM.ZS,Firms with female top manager (% of firms),,"Firms with female top manager refers to the percentage of firms in the private sector who have females as top managers. Top manager refers to the highest ranking manager or CEO of the establishment. This person may be the owner if he/she works as the manager of the firm. The results are based on surveys of more than 100,000 private firms.","World Bank, Enterprise Surveys (http://www.enterprisesurveys.org/).","[{'id': '12', 'value': 'Private Sector'}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
1,IC.FRM.FEMO.ZS,Firms with female participation in ownership (% of firms),,Firms with female participation in ownership are the percentage of firms with a woman among the principal owners.,"World Bank, Enterprise Surveys (http://www.enterprisesurveys.org/).","[{'id': '12', 'value': 'Private Sector'}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
2,SE.ADT.1524.LT.FE.ZS,"Literacy rate, youth female (% of females ages 15-24)",,Youth literacy rate is the percentage of people ages 15-24 who can both read and write with understanding a short simple statement about their everyday life.,UNESCO Institute for Statistics (http://uis.unesco.org/). Data as of June 2022.,"[{'id': '4', 'value': 'Education '}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
3,SE.ADT.1524.LT.FM.ZS,"Literacy rate, youth (ages 15-24), gender parity index (GPI)",,Gender parity index for youth literacy rate is the ratio of females to males ages 15-24 who can both read and write with understanding a short simple statement about their everyday life.,UNESCO Institute for Statistics (http://uis.unesco.org/). Data as of June 2022.,"[{'id': '4', 'value': 'Education '}, {'id': '17', 'value': 'Gender'}, {'id': '15', 'value': 'Social Development '}]",2,World Development Indicators
4,SE.ADT.1524.LT.MA.ZS,"Literacy rate, youth male (% of males ages 15-24)",,Youth literacy rate is the percentage of people ages 15-24 who can both read and write with understanding a short simple statement about their everyday life.,UNESCO Institute for Statistics (http://uis.unesco.org/). Data as of June 2022.,"[{'id': '4', 'value': 'Education '}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
...,...,...,...,...,...,...,...,...
195,SH.MMR.WAGE.ZS,Maternal leave benefits (% of wages paid in covered period),,Maternity leave benefits refers to the total percentage of wages covered by all sources during paid maternity leave.,"World Bank, Women, Business and the Law.","[{'id': '17', 'value': 'Gender'}]",16,Health Nutrition and Population Statistics
196,SH.PAR.LEVE,Length of paid shared parental leave (calendar days),,The indicator refers to parental leave that can be shared amongst the parents as they choose. Parental leave refers to leave available equally to parents—regardless of gender—for the purpose of childcare immediately following maternity and paternity leave OR instead of maternity and paternity leave.,"World Bank: Women, Business and the Law. https://wbl.worldbank.org/","[{'id': '17', 'value': 'Gender'}]",14,Gender Statistics
197,SH.PAR.LEVE.AL,There is paid parental leave (1=yes; 0=no),,The indicator measures whether both parents are legally entitled to some form of full-time paid parental leave either shared between mother and father or as an individual entitlement that each can take regardless of the other.,"World Bank: Women, Business and the Law. https://wbl.worldbank.org/","[{'id': '17', 'value': 'Gender'}]",14,Gender Statistics
198,SH.PAR.LEVE.FE,Length of paid parental leave for mother (calendar days),,The indicator refers to days of parental leave reserved specifically for mothers—through “use-it-or-lose-it” policies or mothers’ quotas; or days of parental leave that employees are individually entitled to.,"World Bank: Women, Business and the Law. https://wbl.worldbank.org/","[{'id

<IPython.core.display.Javascript object>

In [23]:
# Uses callAPI function to get a list of locations
df_locations = callAPI("/locations/")

# Identifies ID code for Western Africa
western_africa_id = df_locations.loc[
    df_locations["name"] == "Western Africa", "id"
].iloc[0]

# Restricts the dataframe to only include geographies from Western Africa
df_locations_for_western_africa_id = df_locations[
    df_locations["parentId"] == western_africa_id
]
df_locations_for_western_africa_id

,id,parentId,name,iso3,iso2,locationTypeId,locationType,longitude,latitude
34,132,914,Cabo Verde,CPV,CV,4,Country,-23.605173,15.120142
53,204,914,Benin,BEN,BJ,4,Country,2.315834,9.307690
73,270,914,Gambia,GMB,GM,4,Country,-15.310139,13.443182
76,288,914,Ghana,GHA,GH,4,Country,-1.023194,7.946527
85,324,914,Guinea,GIN,GN,4,Country,-9.696645,9.945587
99,384,914,Côte d'Ivoire,CIV,CI,4,Country,-5.547080,7.539989
113,430,914,Liberia,LBR,LR,4,Country,-9.429499,6.428055
123,466,914,Mali,MLI,ML,4,Country,-3.996166,17.570692
126,478,914,Mauritania,MRT,MR,4,Country,-10.940835,21.007891
149,562,914,Niger,NER,NE,4,Country,8.081666,17.607788


<IPython.core.display.Javascript object>

In [24]:
# Stores country codes in a list
country_codes_for_western_africa = [
    str(code) for code in df_locations_for_western_africa_id["id"].values
]
country_codes_for_western_africa

['132',
 '204',
 '270',
 '288',
 '324',
 '384',
 '430',
 '466',
 '478',
 '562',
 '566',
 '624',
 '654',
 '686',
 '694',
 '768',
 '854']

<IPython.core.display.Javascript object>

In [25]:
# Converts country code list into a string to be used in later API call
country_selection_string_for_western_africa = ",".join(country_codes_for_western_africa)
country_selection_string_for_western_africa

'132,204,270,288,324,384,430,466,478,562,566,624,654,686,694,768,854'

<IPython.core.display.Javascript object>

In [28]:
# Uses callAPI function to get a list of Family Planning indicators
df_topics = callAPI("/topics/Pop/indicators", topic_list=False)
df_topics

,topicId,topicName,topicShortName,indicatorId,indicatorName,indicatorDisplayName,indicatorShortName,indicatorDescription,dimAge,dimSex,dimVariant,dimCategory,defaultAgeId,defaultSexId,defaultVariantId,defaultCategoryId,unitScaling,formatString,unitLongLabel,unitShortLabel,variableType,sourceId,sourceName,sourceYear,sourceStartYear,sourceEndYear,sourceCitation,sourceUrl
0,1,Population,Pop,53,Crude rate of natural change of population,Crude rate of natural change of population,NatChangeRT,The crude rate of natural change is the ratio of the natural change during the year (live births minus deaths) to the average population in that year. The value is expressed per 1 000 persons.,False,False,True,False,188,3,4,0,1000,##0.0,"per 1,000 population","per 1,000 population",relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
1,1,Population,Pop,41,Female population of reproductive age (15-49 years),Female population of reproductive age (15-49 years),FemPop15_49,Female population of reproductive age (15-49 years),False,False,True,False,31,2,4,99,1,### ### ##0,persons,persons,absolute,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
2,1,Population,Pop,67,Median age of population,Median age of the total population,MedianAgePop,"Age that divides the population in two parts of equal size, that is, there are as many persons with ages above the median as there are with ages below the median. It is expressed as years.",False,False,True,False,188,3,4,0,1,#0.0,years,years,duration,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
3,1,Population,Pop,52,Natural change of population,Natural change of population,NatChange,"The difference between the number of live births and the number of deaths during the year. A positive natural change, also known as natural increase, occurs when live births outnumber deaths. A negative natural change, also named as natural decrease, occurs when live births are less numerous than deaths.",False,False,True,False,188,3,4,0,1,### ### ##0,persons,persons,absolute,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
4,1,Population,Pop,71,Percentage of total population by broad age group,Percentage of total population by broad age group,PopBroadAgeGrpPerc,"Percentage of Total Population by various functional combination of age groups (0-14, 0-17, primary and secondary school ages, 15-24, 15-49, ..., 18+, 50+, etc.). De facto population as of 1 July of the year indicated. Figures are expressed per 100 population.",True,True,True,False,26,3,4,0,1,#0.00,per 100 persons,per 100 persons,relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
5,1,Population,Pop,47,Population by 1-year age groups and sex,Annual population by 1-year age groups and by sex,PopByAge1AndSex,"Annual population by single age and by sex (interpolated data based on 5-year age groups and 5-year periods). De facto population as of 1 July of the year indicated classified by single age (0, 1, 2,.., 99, 100+).",True,True,True,False,42,3,4,0,1,### ### ##0,p

<IPython.core.display.Javascript object>

In [29]:
# Stores indicator codes in a list
indicator_codes = [str(code) for code in df_topics["indicatorId"].values]
indicator_codes

['53', '41', '67', '52', '71', '47', '46', '70', '50', '54', '51', '72', '49']

<IPython.core.display.Javascript object>

In [30]:
# Converts indicator code list into string to be used in later API call
indicator_selection_string = ",".join(indicator_codes)
indicator_selection_string

'53,41,67,52,71,47,46,70,50,54,51,72,49'

<IPython.core.display.Javascript object>

In [31]:
# Calls the API to return the indicator values for the selected indicators and countries.
df = callAPI(
    f"/data/indicators/70/locations/{country_selection_string_for_western_africa}/start/2020/end/2020"
)
df

,locationId,location,iso3,iso2,locationTypeId,indicatorId,indicator,indicatorDisplayName,sourceId,source,revision,variantId,variant,variantShortName,variantLabel,timeId,timeLabel,timeMid,categoryId,category,estimateTypeId,estimateType,estimateMethodId,estimateMethod,sexId,sex,ageId,ageLabel,ageStart,ageEnd,ageMid,value
0,132,Cabo Verde,CPV,CV,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,1,Male,188,Total,0,-1.0,0.0,289745.0
1,132,Cabo Verde,CPV,CV,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,188,Total,0,-1.0,0.0,292895.0
2,132,Cabo Verde,CPV,CV,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1.0,0.0,582640.0
3,132,Cabo Verde,CPV,CV,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,1,Male,42,0,0,1.0,1.0,10091.0
4,132,Cabo Verde,CPV,CV,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,42,0,0,1.0,1.0,9791.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,854,Burkina Faso,BFA,BF,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,53,85+,85,None,85.0,8378.5
2903,854,Burkina Faso,BFA,BF,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,53,85+,85,None,85.0,12413.5
2904,854,Burkina Faso,BFA,BF,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,1,Male,19,90+,90,None,90.0,412.0
2905,854,Burkina Faso,BFA,BF,4,70,Population by age and sex - broad age groups,Population by age and sex - broad age groups,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,19,90+,90,None,90.0,1152.0


<IPython.core.display.Javascript object>

In [36]:
# Finally, filters the returned results to only include median values for All Women, and limits the number of columns retained in the new dataframe.
df2 = df.loc[
    (df["variant"] == "Median") & (df["sex"] == "Both sexes"),
    ["location", "indicator", "variant", "sex", "value"],
]
df2

,location,indicator,variant,sex,value
2,Cabo Verde,Population by age and sex - broad age groups,Median,Both sexes,582640.0
5,Cabo Verde,Population by age and sex - broad age groups,Median,Both sexes,19882.5
8,Cabo Verde,Population by age and sex - broad age groups,Median,Both sexes,51086.5
11,Cabo Verde,Population by age and sex - broad age groups,Median,Both sexes,157563.5
14,Cabo Verde,Population by age and sex - broad age groups,Median,Both sexes,189056.5
...,...,...,...,...,...
2894,Burkina Faso,Population by age and sex - broad age groups,Median,Both sexes,314145.0
2897,Burkina Faso,Population by age and sex - broad age groups,Median,Both sexes,152671.5
2900,Burkina Faso,Population by age and sex - broad age groups,Median,Both sexes,53150.5
2903,Burkina Faso,Population by age and sex - broad age groups,Median,Both sexes,12413.5


<IPython.core.display.Javascript object>

In [38]:
base_url_UNPD = "https://population.un.org/dataportalapi/api/v1"

<IPython.core.display.Javascript object>

In [40]:
target = base_url_UNPD + "/topics/"  # Define target URL
response = requests.get(target)  # Call the API
j = response.json()  # Convert response into JSON object
df = pd.json_normalize(j["data"])  # convert JSON to data frame
df

,id,name,shortName,sortOrder
0,0,Not applicable,NA,0
1,1,Population,Pop,1
2,2,Fertility,Fert,2
3,3,Mortality,Mort,3
4,4,International Migration,iMigration,4
5,5,Family Planning,FP,5
6,6,Marital Status,MarStat,6
7,7,All Components,All,7
8,8,Child Mortality,IGME,8
9,9,Maternal Mortality,MMEIG,9


<IPython.core.display.Javascript object>

In [41]:
target = base_url_UNPD + "/locations/"  # Define target URL
response = requests.get(target)  # Call the API
j = response.json()  # Convert response into JSON object
df = pd.json_normalize(j["data"])  # convert JSON to data frame
df

,id,parentId,name,iso3,iso2,locationTypeId,locationType,longitude,latitude
0,4,5501,Afghanistan,AFG,AF,4,Country,67.709953,33.939110
1,8,925,Albania,ALB,AL,4,Country,20.168331,41.153332
2,12,912,Algeria,DZA,DZ,4,Country,1.659626,28.033886
3,16,957,American Samoa,ASM,AS,4,Country,-170.696182,-14.306021
4,20,925,Andorra,AND,AD,4,Country,1.521801,42.506287
...,...,...,...,...,...,...,...,...,...
95,368,922,Iraq,IRQ,IQ,4,Country,43.679291,33.223190
96,372,924,Ireland,IRL,IE,4,Country,-8.243890,53.412910
97,376,922,Israel,ISR,IL,4,Country,34.851612,31.046051
98,380,925,Italy,ITA,IT,4,Country,12.567380,41.871941


<IPython.core.display.Javascript object>

In [44]:
target = base_url_UNPD + "/locationsWithAggregates/"  # Define target URL
response = requests.get(target)  # Call the API
j = response.json()  # Convert response into JSON object
df = pd.json_normalize(j)  # convert JSON to data frame
df

,Id,Name,Iso2,Iso3,Longitude,Latitude,Region,SubRegion,SpecialGrouping,WorldBankIncomeGroup,UNDevelopmentGroup,SDGRegion
0,4,Afghanistan,AF,AFG,67.709953,33.939110,Asia,Southern Asia,Landlocked developing countries (LLDCs),Low-income countries,Least developed countries,Central and Southern Asia
1,8,Albania,AL,ALB,20.168331,41.153332,Europe,Southern Europe,NaN,Upper-middle-income countries,Developed regions,Europe and Northern America
2,12,Algeria,DZ,DZA,1.659626,28.033886,Africa,Northern Africa,NaN,Lower-middle-income countries,Other developing regions,Northern Africa and Western Asia
3,16,American Samoa,AS,ASM,-170.696182,-14.306021,Oceania,Polynesia,Small island developing States (SIDS),Upper-middle-income countries,Other developing regions,Oceania (excluding Australia and New Zealand)
4,20,Andorra,AD,AND,1.521801,42.506287,Europe,Southern Europe,NaN,High-income countries,Developed regions,Europe and Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...
95,364,Iran (Islamic Republic of),IR,IRN,53.688046,32.427910,Asia,Southern Asia,NaN,Lower-middle-income countries,Other developing regions,Central and Southern Asia
96,368,Iraq,IQ,IRQ,43.679291,33.223190,Asia,Western Asia,NaN,Upper-middle-income countries,Other developing regions,Northern Africa and Western Asia
97,372,Ireland,IE,IRL,-8.243890,53.412910,Europe,Northern Europe,NaN,High-income countries,Developed regions,Europe and Northern America
98,376,Israel,IL,ISR,34.851612,31.046051,Asia,Western Asia,NaN,High-income countries,Other developing regions,Northern Africa and Western Asia


<IPython.core.display.Javascript object>

In [45]:
base_url_WB = "http://api.worldbank.org/v2/"

<IPython.core.display.Javascript object>

In [54]:
target = base_url_WB + "/topic?format=json"  # Define World Bank API URL
response = requests.get(target)
j = response.json()
df = pd.json_normalize(j[1])
df

,id,value,sourceNote
0,1,Agriculture & Rural Development,"For the 70 percent of the world's poor who live in rural areas, agriculture is the main source of income and employment. But depletion and degradation of land and water pose serious challenges to producing enough food and other agricultural products to sustain livelihoods here and meet the needs of urban populations. Data presented here include measures of agricultural inputs, outputs, and productivity compiled by the UN's Food and Agriculture Organization."
1,2,Aid Effectiveness,"Aid effectiveness is the impact that aid has in reducing poverty and inequality, increasing growth, building capacity, and accelerating achievement of the Millennium Development Goals set by the international community. Indicators here cover aid received as well as progress in reducing poverty and improving education, health, and other measures of human welfare."
2,3,Economy & Growth,"Economic growth is central to economic development. When national income grows, real people benefit. While there is no known formula for stimulating economic growth, data can help policy-makers better understand their countries' economic situations and guide any work toward improvement. Data here covers measures of economic growth, such as gross domestic product (GDP) and gross national income (GNI). It also includes indicators representing factors known to be relevant to economic growth, such as capital stock, employment, investment, savings, consumption, government spending, imports, and exports."
3,4,Education,"Education is one of the most powerful instruments for reducing poverty and inequality and lays a foundation for sustained economic growth. The World Bank compiles data on education inputs, participation, efficiency, and outcomes. Data on education are compiled by the United Nations Educational, Scientific, and Cultural Organization (UNESCO) Institute for Statistics from official responses to surveys and from reports provided by education authorities in each country."
4,5,Energy & Mining,"The world economy needs ever-increasing amounts of energy to sustain economic growth, raise living standards, and reduce poverty. But today's trends in energy use are not sustainable. As the world's population grows and economies become more industrialized, nonrenewable energy sources will become scarcer and more costly. Data here on energy production, use, dependency, and efficiency are compiled by the World Bank from the International Energy Agency and the Carbon Dioxide Information Analysis Center."
5,6,Environment,"Natural and man-made environmental resources – fresh water, clean air, forests, grasslands, marine resources, and agro-ecosystems – provide sustenance and a foundation for social and economic development. The need to safeguard these resources crosses all borders. Today, the World Bank is one of the key promoters and financiers of environmental upgrading in the developing world. Data here cover forests, biodiversity, emissions, and pollution. Other indicators relevant to the environment are found under data pages for Agriculture & Rural Development, Energy & Mining, Infrastructure, and Urban Development."
6,7,Financial Sector,"An economy's financial markets are critical to its overall development. Banking systems and stock markets enhance growth, the main factor in poverty reduction. Strong financial systems provide reliable and accessible information that lowers transaction costs, which in turn bolsters resource allocation and economic growth. Indicators here include the size and liquidity of stock markets; the accessibility, stability, and efficiency of financial systems; and international migration and workers\ remittances, which affect growth and social welfare in both sending and receiving countries."
7,8,Health,"Improving health is central to the Millennium Development Goals, and the public sector is the main provider of health care in developing countries. To reduce inequities, many countries have e

<IPython.core.display.Javascript object>

In [59]:
target = (
    base_url_WB + "/topic/17/indicator?format=json&per_page=200"
)  # Define World Bank API URL

response = requests.get(target)
j = response.json()
df = pd.json_normalize(j[1])
df

,id,name,unit,sourceNote,sourceOrganization,topics,source.id,source.value
0,IC.FRM.FEMM.ZS,Firms with female top manager (% of firms),,"Firms with female top manager refers to the percentage of firms in the private sector who have females as top managers. Top manager refers to the highest ranking manager or CEO of the establishment. This person may be the owner if he/she works as the manager of the firm. The results are based on surveys of more than 100,000 private firms.","World Bank, Enterprise Surveys (http://www.enterprisesurveys.org/).","[{'id': '12', 'value': 'Private Sector'}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
1,IC.FRM.FEMO.ZS,Firms with female participation in ownership (% of firms),,Firms with female participation in ownership are the percentage of firms with a woman among the principal owners.,"World Bank, Enterprise Surveys (http://www.enterprisesurveys.org/).","[{'id': '12', 'value': 'Private Sector'}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
2,SE.ADT.1524.LT.FE.ZS,"Literacy rate, youth female (% of females ages 15-24)",,Youth literacy rate is the percentage of people ages 15-24 who can both read and write with understanding a short simple statement about their everyday life.,UNESCO Institute for Statistics (http://uis.unesco.org/). Data as of June 2022.,"[{'id': '4', 'value': 'Education '}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
3,SE.ADT.1524.LT.FM.ZS,"Literacy rate, youth (ages 15-24), gender parity index (GPI)",,Gender parity index for youth literacy rate is the ratio of females to males ages 15-24 who can both read and write with understanding a short simple statement about their everyday life.,UNESCO Institute for Statistics (http://uis.unesco.org/). Data as of June 2022.,"[{'id': '4', 'value': 'Education '}, {'id': '17', 'value': 'Gender'}, {'id': '15', 'value': 'Social Development '}]",2,World Development Indicators
4,SE.ADT.1524.LT.MA.ZS,"Literacy rate, youth male (% of males ages 15-24)",,Youth literacy rate is the percentage of people ages 15-24 who can both read and write with understanding a short simple statement about their everyday life.,UNESCO Institute for Statistics (http://uis.unesco.org/). Data as of June 2022.,"[{'id': '4', 'value': 'Education '}, {'id': '17', 'value': 'Gender'}]",2,World Development Indicators
...,...,...,...,...,...,...,...,...
195,SH.MMR.WAGE.ZS,Maternal leave benefits (% of wages paid in covered period),,Maternity leave benefits refers to the total percentage of wages covered by all sources during paid maternity leave.,"World Bank, Women, Business and the Law.","[{'id': '17', 'value': 'Gender'}]",16,Health Nutrition and Population Statistics
196,SH.PAR.LEVE,Length of paid shared parental leave (calendar days),,The indicator refers to parental leave that can be shared amongst the parents as they choose. Parental leave refers to leave available equally to parents—regardless of gender—for the purpose of childcare immediately following maternity and paternity leave OR instead of maternity and paternity leave.,"World Bank: Women, Business and the Law. https://wbl.worldbank.org/","[{'id': '17', 'value': 'Gender'}]",14,Gender Statistics
197,SH.PAR.LEVE.AL,There is paid parental leave (1=yes; 0=no),,The indicator measures whether both parents are legally entitled to some form of full-time paid parental leave either shared between mother and father or as an individual entitlement that each can take regardless of the other.,"World Bank: Women, Business and the Law. https://wbl.worldbank.org/","[{'id': '17', 'value': 'Gender'}]",14,Gender Statistics
198,SH.PAR.LEVE.FE,Length of paid parental leave for mother (calendar days),,The indicator refers to days of parental leave reserved specifically for mothers—through “use-it-or-lose-it” policies or mothers’ quotas; or days of parental leave that employees are individually entitled to.,"World Bank: Women, Business and the Law. https://wbl.worldbank.org/","[{'id

<IPython.core.display.Javascript object>

In [60]:

country_iso3 = "SEN" # Set the desired country
indicator_code_WB = "SE.PRM.NENR" # Set the desired indicator

target = base_url_WB + f"/country/{country_iso3}/indicator/{indicator_code_WB}?format=json" # Define the target url

response = requests.get(target) # Call WB API
j = response.json() # Create JSON object
pages = j[0]['pages'] # Identify number of pages in response

#Convert first page into a DataFrame
df_WB = pd.json_normalize(j[1])

#Loop through pages and append results to DataFrame
for page in range(2,pages+1):
    target = base_url_WB + f"/country/{country_iso3}/indicator/{indicator_code_WB}?format=json&page={page}"
    response = requests.get(target)
    j = response.json()
    df_temp = pd.json_normalize(j[1])
    df_WB = df_WB.append(df_temp)

# Verify that the length of the DataFrame is equal to the number of observations in the API response. Otherwise, raise an error.
assert len(df_WB)==j[0]['total'], "DataFrame observations do not match total number of records in response"


/tmp/ipykernel_4811/2416665952.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_WB = df_WB.append(df_temp)


<IPython.core.display.Javascript object>

### Extra

In [68]:
# Define a function that will take a relative path as an input, call the API, and return a dataframe
def callAPI(relative_path: str, topic_list: bool = False) -> pd.DataFrame:
    base_url = "https://population.un.org/dataportalapi/api/v1"
    target = (
        base_url + relative_path
    )  # Query string parameters may be appended here or directly in the provided relative path
    # Calls the API
    response = requests.get(target)
    # Reformats response into a JSON object
    j = response.json()
    # The block below will deal with paginated results.
    # If results not paginated, this will be skipped.
    try:
        # If results are paginated, they are transformed into a python dictionary.
        # The data may be accessed using the 'data' key of the dictionary.
        df = pd.json_normalize(j["data"])
        # As long as the nextPage key of the dictionary contains an address for the next API call, the function will continue to call the API and append the results to the dataframe.
        while j["nextPage"] is not None:
            response = requests.get(j["nextPage"])
            j = response.json()
            df_temp = pd.json_normalize(j["data"])
            df = pd.concat([df, df_temp], ignore_index=True)
    except:
        if topic_list:
            df = pd.json_normalize(j)
        else:
            df = pd.DataFrame(j)
    return df

<IPython.core.display.Javascript object>

In [69]:
new_df = callAPI("/locationsWithAggregates/")
new_df

,Id,Name,Iso2,Iso3,Longitude,Latitude,Region,SubRegion,SpecialGrouping,WorldBankIncomeGroup,UNDevelopmentGroup,SDGRegion
0,4,Afghanistan,AF,AFG,67.709953,33.939110,Asia,Southern Asia,Landlocked developing countries (LLDCs),Low-income countries,Least developed countries,Central and Southern Asia
1,8,Albania,AL,ALB,20.168331,41.153332,Europe,Southern Europe,NaN,Upper-middle-income countries,Developed regions,Europe and Northern America
2,12,Algeria,DZ,DZA,1.659626,28.033886,Africa,Northern Africa,NaN,Lower-middle-income countries,Other developing regions,Northern Africa and Western Asia
3,16,American Samoa,AS,ASM,-170.696182,-14.306021,Oceania,Polynesia,Small island developing States (SIDS),Upper-middle-income countries,Other developing regions,Oceania (excluding Australia and New Zealand)
4,20,Andorra,AD,AND,1.521801,42.506287,Europe,Southern Europe,NaN,High-income countries,Developed regions,Europe and Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...
95,364,Iran (Islamic Republic of),IR,IRN,53.688046,32.427910,Asia,Southern Asia,NaN,Lower-middle-income countries,Other developing regions,Central and Southern Asia
96,368,Iraq,IQ,IRQ,43.679291,33.223190,Asia,Western Asia,NaN,Upper-middle-income countries,Other developing regions,Northern Africa and Western Asia
97,372,Ireland,IE,IRL,-8.243890,53.412910,Europe,Northern Europe,NaN,High-income countries,Developed regions,Europe and Northern America
98,376,Israel,IL,ISR,34.851612,31.046051,Asia,Western Asia,NaN,High-income countries,Other developing regions,Northern Africa and Western Asia


<IPython.core.display.Javascript object>